**Oбучение с подкреплением**

Евгений Борисов borisov.e@solarl.ru

In [1]:
import numpy as np
import gym

In [2]:
env = gym.make('CartPole-v1')
# ENV_NAME = 'BreakoutDeterministic-v4'

In [3]:
s = env.reset()
env.render()
for t in range(150):
    a = env.action_space.sample()
    sn,r,d,_ = env.step(a)
    env.render()
    print('%d: %.1f %d'%(t,r,d) )
    s=sn
    #if d: break
env.close()  

0: 1.0 0
1: 1.0 0
2: 1.0 0
3: 1.0 0
4: 1.0 0
5: 1.0 0
6: 1.0 0
7: 1.0 0
8: 1.0 0
9: 1.0 0
10: 1.0 0
11: 1.0 0
12: 1.0 0
13: 1.0 0
14: 1.0 0
15: 1.0 0
16: 1.0 0
17: 1.0 0
18: 1.0 0
19: 1.0 0
20: 1.0 0
21: 1.0 1


/usr/lib/python3.9/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


22: 0.0 1
23: 0.0 1
24: 0.0 1
25: 0.0 1
26: 0.0 1
27: 0.0 1
28: 0.0 1
29: 0.0 1
30: 0.0 1
31: 0.0 1
32: 0.0 1
33: 0.0 1
34: 0.0 1
35: 0.0 1
36: 0.0 1
37: 0.0 1
38: 0.0 1
39: 0.0 1
40: 0.0 1
41: 0.0 1
42: 0.0 1
43: 0.0 1
44: 0.0 1
45: 0.0 1
46: 0.0 1
47: 0.0 1
48: 0.0 1
49: 0.0 1
50: 0.0 1
51: 0.0 1
52: 0.0 1
53: 0.0 1
54: 0.0 1
55: 0.0 1
56: 0.0 1
57: 0.0 1
58: 0.0 1
59: 0.0 1
60: 0.0 1
61: 0.0 1
62: 0.0 1
63: 0.0 1
64: 0.0 1
65: 0.0 1
66: 0.0 1
67: 0.0 1
68: 0.0 1
69: 0.0 1
70: 0.0 1
71: 0.0 1
72: 0.0 1
73: 0.0 1
74: 0.0 1
75: 0.0 1
76: 0.0 1
77: 0.0 1
78: 0.0 1
79: 0.0 1
80: 0.0 1
81: 0.0 1
82: 0.0 1
83: 0.0 1
84: 0.0 1
85: 0.0 1
86: 0.0 1
87: 0.0 1
88: 0.0 1
89: 0.0 1
90: 0.0 1
91: 0.0 1
92: 0.0 1
93: 0.0 1
94: 0.0 1
95: 0.0 1
96: 0.0 1
97: 0.0 1
98: 0.0 1
99: 0.0 1
100: 0.0 1
101: 0.0 1
102: 0.0 1
103: 0.0 1
104: 0.0 1
105: 0.0 1
106: 0.0 1
107: 0.0 1
108: 0.0 1
109: 0.0 1
110: 0.0 1
111: 0.0 1
112: 0.0 1
113: 0.0 1
114: 0.0 1
115: 0.0 1
116: 0.0 1
117: 0.0 1
118: 0.0 1
119: 0.0 1


---

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
# from keras.layers import Dropout
# from keras.optimizers import SGD

In [5]:
def create_model(env):
    md = Sequential()
    md.add(Dense(256, activation='relu',input_shape=env.observation_space.shape))
    md.add(Dense(256, activation='relu'))
    md.add(Dense(env.action_space.n))
    md.compile(loss='mean_squared_error', optimizer='SGD')
    
    #md.add(Dense(env.action_space.n, activation='softmax'))
    #md.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4))
    return md

In [6]:
model = create_model(env)
model_target = create_model(env)

In [7]:
HISTORY_DEEP = 2000 # глубина истории переходов
BATCH_SIZE = 32 # размер батча для обучения модели

GAMMA = 0.85 # коэффициент для расчёта оценки reward
TAU = 0.125 # определяет различие весов основной и таргет моделей

# вероятность использования случайного действия при обучении
EPSILON_MAX = 1.0 # в начальном состоянии используем случайный выбор действия
EPSILON_MIN = 0.05 # когда модель начала обучаться позволяем ей оценивать действия
EPSILON_DECAY = 0.995 # шаг изменения вероятности

MAX_EPOCH = 100 # максимальное количество эпох обучения
MAX_STEP = 256  # максимальное количество шагов в эпохе (длительность одной игры)

In [8]:
def reshape_state(s): return s[np.newaxis,:]

In [9]:
# выбор действия
def get_action(model,state,eps=EPSILON_MAX): # с вероятностью eps выбираем случайное действие
    if np.random.rand()<eps: return env.action_space.sample(), 1
    return np.argmax( model.predict( state )[0] ), 0

In [10]:
def replay(model, model_target, history, batch_size=BATCH_SIZE,gamma=GAMMA):

    if len(history) < batch_size: return
    
    samples = [ history[i] for i in np.random.permutation(len(history))[:batch_size] ] 

    for s in samples:
        target = model_target.predict( s['state0'] )
        
        target[0][ s['act'] ] = s['reward'] 
        
        if s['isdone']:
            target[0][ s['act'] ] = s['reward']
        else:
            target[0][ s['act'] ] = s['reward'] + gamma*max(model_target.predict(s['state1'])[0])
            
        # model.fit( s['state0'], target, epochs=1, verbose=0)
        model.train_on_batch(s['state0'], target)

In [11]:
def target_train(model, model_target,tau=TAU):
    weights = model.get_weights()
    target_weights = model_target.get_weights()
    for i in range(len(target_weights)):
        target_weights[i] = weights[i] * tau + target_weights[i] * (1. - tau)
    model_target.set_weights(target_weights)

In [12]:
%%time

history = []

epsilon = EPSILON_MAX # вероятность использования случайного действия при обучении

for e in range(MAX_EPOCH):
    # инициализируем среду
    state0 = env.reset() 
    state0 = reshape_state(state0)
    for st in range(MAX_STEP):
        
        # выбираем действие
        act, is_act_rand = get_action(model,state0,epsilon)
        # применяем действие
        state1, reward, done, _ = env.step(act)
        state1 = reshape_state(state1)
    
        # сохраняем историю
        history.append({'state0':state0,'state1':state1,'act':act,'reward':reward,'isdone':done})
        history = history[-HISTORY_DEEP:] # ограничиваем историю

        # коррекция основной модели
        replay(model, model_target, history)
        
        # коррекция таргет модели
        target_train(model, model_target)
                
        if done: break
        state0 = state1
        epsilon = max(EPSILON_MIN, epsilon*EPSILON_DECAY)
                
    print('%d/%d : %d steps'%(e+1,MAX_EPOCH,st+1))  
    
    if st>200: 
        print('STEP BOUND!')
        break

1/100 : 23 steps
2/100 : 14 steps
3/100 : 45 steps
4/100 : 32 steps
5/100 : 46 steps
6/100 : 73 steps
7/100 : 129 steps
8/100 : 96 steps
9/100 : 98 steps
10/100 : 22 steps
11/100 : 187 steps
12/100 : 136 steps
13/100 : 226 steps
STEP BOUND!
CPU times: user 52min 8s, sys: 53.7 s, total: 53min 2s
Wall time: 51min 33s


In [13]:
# len(history)
# history

In [20]:
model.save('CartPole-%d.model'%(st))

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: CartPole-225.model/assets


---

In [15]:
# from tensorflow.keras.models import load_model
# model = load_model('CartPole-12.model')

In [18]:
s = env.reset()[np.newaxis,:]
env.render()
#while True:
for t in range(350):    
    a = np.argmax( model.predict(s)[0] )   
    s,r,d,_ = env.step(a)
    s=s[np.newaxis,:]
    env.render()
    if d: 
        print(t) 
        break

In [19]:
env.close()    